In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [17, 6]
from IPython.display import Audio
from ipywidgets import HTML

In [ ]:
from pathlib import Path

import torchcrepe
import torch
import librosa

In [ ]:
# file = Path('/home/kureta/Music/violin/Violin Samples/yee_amazing_grace#7.wav')
file = Path('/home/kureta/Music/violin/Violin Samples/yee_amazing_grace.wav')

In [ ]:
sample_rate = 48000
# Load audio
audio, sr = librosa.load(file, sample_rate)
audio = torch.from_numpy(audio.astype('float32')).unsqueeze(0)

# Here we'll use a 10 millisecond hop length
hop_length = 480

# Provide a sensible frequency range for your domain (upper limit is 2006 Hz)
# This would be a reasonable range for speech
fmin = 190
fmax = 2800

# Select a model capacity--one of "tiny" or "full"
model = 'full'

# Choose a device to use for inference
device = 'cuda'

# Pick a batch size that doesn't cause memory errors on your gpu
batch_size = 2048

# Compute pitch using first gpu
pitch, periodicity = torchcrepe.predict(audio,
                           sr,
                           hop_length,
                           fmin,
                           fmax,
                           model,
                           batch_size=batch_size,
                           device=device,
                           decoder=torchcrepe.decode.viterbi,
                           return_periodicity=True)

In [ ]:
# Filter silence
periodicity = torchcrepe.threshold.Silence(-90.)(periodicity,
                                                 audio,
                                                 sr,
                                                 hop_length)

# We'll use a 15 millisecond window assuming a hop length of 5 milliseconds
win_length = 3

# Median filter noisy confidence value
periodicity = torchcrepe.filter.median(periodicity, win_length)

# Remove inharmonic regions
pitch = torchcrepe.threshold.At(.21)(pitch, periodicity)

# Optionally smooth pitch to remove quantization artifacts
pitch = torchcrepe.filter.mean(pitch, win_length)

In [ ]:
loudness = torchcrepe.loudness.a_weighted(audio, sample_rate, hop_length)

In [ ]:
plt.plot(periodicity[0])

In [ ]:
plt.plot(loudness[0])

In [ ]:
plt.plot(pitch[0])

In [ ]:
Audio(data=audio, rate=sample_rate)

In [ ]:
pitch.isnan().any()

In [ ]:
331433/